# EGN6217 P2/P3 Gradio

In [ ]:
!pip install transformers diffusers datasets jiwer gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.0 MB/s eta 0:00:00


In [ ]:
import os
import re
import ast
import hashlib
import warnings
import random
import requests
import torch
import torch.nn.functional as F
import torchaudio
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import GradScaler, autocast
from torchvision import transforms
from datasets import load_dataset
from tqdm import tqdm
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, CLIPTokenizer
from diffusers import StableDiffusionPipeline
from PIL import Image
from io import BytesIO
import jiwer
import gradio as gr
from IPython.display import clear_output, Audio, display
from google.colab import drive
import textwrap

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# setup Wav2Vec2 environment
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h", do_normalize=True)
wav2vec2_model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/Colab Data/egn6217-p3/wav2vec2_model")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
# setup Stable Diffusion v1.5 environment

warnings.filterwarnings("ignore")

# tokenizer, model, loss function
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14") # in Stable Diffusion v1.5 architecture, imported separately here for collate_fn
diffusion_pipe = StableDiffusionPipeline.from_pretrained("/content/drive/MyDrive/Colab Data/egn6217-p3/diffusion_pipe", torch_dtype=torch.float32) ######################## torch.float16
loss_fn = F.mse_loss

# Dataset and DataLoader

class FaceCaption15MDataset(Dataset): # dynamic loading for training
    def __init__(self, xs, ts_paths):
        # set img stuff
        self.xs = xs             # img descriptions
        self.ts_paths = ts_paths # img paths

    def __len__(self):
        return len(self.xs)

    def __getitem__(self, idx):
        # load img stuff
        x = self.xs[idx]                   # img description
        t = torch.load(self.ts_paths[idx]) # img
        return x, t

def collate_fn(batch): # dynamic batch-wise tokenization
    xs, ts = zip(*batch)
    xs = tokenizer(xs, padding="longest", truncation=True, max_length=77, return_tensors="pt")
    ts = torch.stack(ts, dim=0)
    return xs, ts

test_dataset = torch.load("/content/drive/MyDrive/Colab Data/egn6217-p3/test_dataset.pth", weights_only=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8, collate_fn=collate_fn, pin_memory=True) ######################## high n_workers high pin_memory = faster compute but more used memory

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
def make_test_gradio(processor, wav2vec2_model, diffusion_pipe):

  def test_gradio(audio):
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

      # speech-to-text

      # preprocess audio
      sr, audio = audio
      audio = processor(audio.squeeze().astype(np.float32), padding=True, sampling_rate=16000, return_attention_mask=True, return_tensors="pt")
      audio_att_mask = audio["attention_mask"].to(device)
      audio = audio["input_values"].to(device)

      # forward propagation
      # wav2vec2_model.to(device)
      wav2vec2_model.eval()
      outputs = wav2vec2_model(audio, attention_mask=audio_att_mask)

      # decode predictions
      pred_ids = torch.argmax(outputs.logits, dim=-1)
      pred_transcript = processor.batch_decode(pred_ids, skip_special_tokens=True, group_tokens=True)[0]

      # text-to-image

      # diffusion_pipe.to(device)
      img = diffusion_pipe(prompt=pred_transcript, guidance_scale=15).images[0]

      return pred_transcript, img

  return test_gradio

# pre-move models to GPU and make gradio function w/ models in scope

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wav2vec2_model = wav2vec2_model.to(device)
diffusion_pipe = diffusion_pipe.to(device)

test_gradio = make_test_gradio(processor, wav2vec2_model, diffusion_pipe)

In [ ]:
interface = gr.Interface(
    fn=test_gradio,
    inputs=gr.Audio(
        sources=["microphone"],
        type="numpy",
        label="Record your audio",
        waveform_options=gr.WaveformOptions(sample_rate=16000)
    ),
    outputs=[
        gr.Textbox(label="Transcription"),
        gr.Image(label="Generated Image")
    ],
    title="Speech-to-Image Generation",
    description="Record audio to generate both a transcript and an AI-generated image from the transcript."
)

# launch the interface
interface.launch(inline=False, inbrowser=True, debug=True)